# 지도학습 평가

[머신러닝 모델 평가](https://data-gardner.tistory.com/16)

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import mglearn

plt.rc('figure', figsize=(10, 6))

from matplotlib import rcParams
rcParams['font.family'] = 'New Gulim'
rcParams['font.size'] = 10
rcParams['axes.unicode_minus'] = False

# 1 회귀 평가

### 1.1 회귀 평가 지표
평가 지표 | 설명 | 수식
:--- |:---|:---
MAE | Mean Absolute Error, 측정값과 예측값의 차이를 절대값으로 변환해 평균을 구함 | $$MAE=\frac {1} {N} \sum_{i}^N |Y_{i} - \hat Y_{i}|$$
MSE | Mean Squared Error, 측정값과 예측값의 차이를 제곱해 평균을 구함 | $$MSE=\frac {1} {N} \sum_{i}^N (Y_{i} - \hat Y_{i})^2$$
MSLE | Mean Squared Log Error, 측정값과 예측값에 로그를 취한 후 차이를 제곱해 평균을 구함 | $$MSLE=\frac {1} {N} \sum_{i}^N (\log (Y_{i}+1) - \log (\hat Y_{i}+1))^2$$
RMSE | Root Mean Squared Error, MSE값에 루트를 씌워 스케일을 맞춤 | $$RMSE=\sqrt {\frac {1} {N} \sum_{i}^N (Y_{i} - \hat Y_{i})^2}$$
$R^2$ | 측정값의 분산 대비 예측값의 분산 비율을 지표로 하며, 1에 가까울 수록 모델의 설명력이 높음 | $$R^2 = \frac {SSE} {SST} = 1 - \frac {SSR} {SST}$$

### 1.2 선형 회귀 적용 - 보스턴 집값 예측

#### 1.2.1 데이터 로딩

In [ ]:
# 데이터 로딩
df = pd.read_csv('data/boston.csv')
X = df.drop('target', axis=1).values
y = df['target'].values

df

#### 1.2.2 데이터 탐색

In [ ]:
fig, axs = plt.subplots(figsize=(16,8) , ncols=4 , nrows=2)

col_list = ['RM','ZN','INDUS','NOX','AGE','PTRATIO','LSTAT','RAD']

for i, feature in enumerate(col_list):
    row = int(i/4)
    col = i%4
    sns.regplot(x=feature, y='target', data=df , ax=axs[row][col])
    
plt.show()

#### 1.2.3 선형 회귀 분석

In [ ]:
# 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

In [ ]:
# Linear Regression
from sklearn.linear_model import LinearRegression

# 모델 생성
model = LinearRegression(fit_intercept=True)

In [ ]:
# 모델 학습
model.fit(X_train, y_train)

In [ ]:
# 결과 예측
pred = model.predict(X_test)
pred

#### 1.2.4 선형 회귀 평가

In [ ]:
from sklearn import metrics

In [ ]:
# Coefficient
model.coef_

In [ ]:
# intercept
model.intercept_

In [ ]:
# MAE(Mean Absolute Error)
metrics.mean_absolute_error(y_test, pred)

In [ ]:
# MSE(Mean Squared Error)
metrics.mean_squared_error(y_test, pred)

In [ ]:
# RMSE(Root Mean Squared Error)
metrics.mean_squared_error(y_test, pred)**0.5

In [ ]:
# R square
metrics.r2_score(y_train, model.predict(X_train))

In [ ]:
# MAPE
def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - pred) / y_test)) * 100 
    
MAPE(y_test, pred)

In [ ]:
# Regression plot
plt.scatter(y_test, pred)
plt.xlabel('y_test')
plt.ylabel('pred')
plt.show()

#### 1.2.5 cross_val_score

In [ ]:
from sklearn.model_selection import cross_val_score
neg_scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=5)

scores  = np.sqrt(-1 * neg_scores)

print('교차 검증 점수:', scores)
print('교차 검증 평균 점수: {:.2f}'.format(scores.mean()))

# 2 분류 평가
- Accuracy
- Precision
- Recall
- F1-score
- ROC Curve
- AUC

### 2.1 불균형 데이터 셋

In [ ]:
# 데이터 로딩
from sklearn.datasets import load_digits
digits = load_digits()

In [ ]:
# 불균형 데이터 셋 생성
X = digits.data
y = digits.target == 9

In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

#### 2.1.1 더미 분류기 1 - 빈도 기반

In [ ]:
from sklearn.dummy import DummyClassifier

model_dummy1 = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
pred_dummy1  = model_dummy1.predict(X_test)

In [ ]:
model_dummy1.score(X_test, y_test)

#### 2.1.2 더미 분류기 2 - 무작위

In [ ]:
model_dummy2 = DummyClassifier().fit(X_train, y_train)
pred_dummy2  = model_dummy2.predict(X_test)

In [ ]:
model_dummy2.score(X_test, y_test)

#### 2.1.3 결정 트리

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model_tree = DecisionTreeClassifier(max_depth=2).fit(X_train, y_train)
pred_tree = model_tree.predict(X_test)

In [ ]:
model_tree.score(X_test, y_test)

#### 2.1.4 Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

model_logreg = LogisticRegression(C=0.1, max_iter=1000).fit(X_train, y_train)
pred_logreg = model_logreg.predict(X_test)

In [ ]:
model_logreg.score(X_test, y_test)

### 2.2 오차 행렬(Confusion Matrix)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, pred_logreg)

print('Confusion Matrix\n', confusion)

In [ ]:
pd.crosstab(y_test, pred_logreg, margins=True)

In [ ]:
mglearn.plots.plot_confusion_matrix_illustration()

In [ ]:
mglearn.plots.plot_binary_confusion_matrix()

#### 2.2.1 각 모델별 오차 행렬(Confusion Matrix)

In [ ]:
print('1. 빈도 기반 더미 모델')
print(confusion_matrix(y_test, pred_dummy1))

print('\n2. 무작위 더미 모델')
print(confusion_matrix(y_test, pred_dummy2))

print('\n3. 결정 트리')
print(confusion_matrix(y_test, pred_tree))

print('\n4. 로지스틱 회귀')
print(confusion_matrix(y_test, pred_logreg))

### 2.3 분류 평가 지표

- 정확도(Accuracy)
$$\text{Accuracy} = \frac{\text{TP} + \text{TN}}{\text{TP} + \text{TN} + \text{FP} + \text{FN}}$$

- 정밀도(Precision)
$$\text{Precision} = \frac{\text{TP}}{\text{TP} + \text{FP}}$$

- 재현율(Recall)
$$\text{Recall} = \frac{\text{TP}}{\text{TP} + \text{FN}}$$

- F1 스코어
$$\text{F1-score} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$$

In [ ]:
# Accuracy
metrics.accuracy_score(y_test,pred_logreg)

In [ ]:
# Precision
metrics.precision_score(y_test, pred_logreg, average='weighted')

In [ ]:
# Recall
metrics.recall_score(y_test, pred_logreg, average='weighted')

In [ ]:
# F1-score
metrics.f1_score(y_test, pred_logreg, average='weighted')

#### 2.3.1 각 모델별 F1-score

In [ ]:
from sklearn.metrics import f1_score

print('1. 빈도 기반 더미 모델: {:.2f}'.format(f1_score(y_test, pred_dummy1)))
print('2. 무작위 더미 모델: {:.2f}'.format(f1_score(y_test, pred_dummy2)))
print('3. 결정 트리: {:.2f}'.format(f1_score(y_test, pred_tree)))
print('4. 로지스틱 회귀: {:.2f}'.format(f1_score(y_test, pred_logreg)))

#### 2.3.2 각 모델별 Classification Report

In [ ]:
# 1. 빈도 기반 더미 모델
from sklearn.metrics import classification_report
print(classification_report(y_test, pred_dummy1, target_names=['9 아님', '9'], zero_division=0))

In [ ]:
# 2. 무작위 더미 모델
print(classification_report(y_test, pred_dummy2, target_names=['9 아님', '9'], zero_division=0))

In [ ]:
# 3. 결정 트리
print(classification_report(y_test, pred_tree, target_names=['9 아님', '9']))

In [ ]:
# 4. 로지스틱 회귀
print(classification_report(y_test, pred_logreg, target_names=['9 아님', '9']))

#### 2.3.3 임계값 조정

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.svm import SVC

In [ ]:
# 데이터 로딩
X, y = make_blobs(n_samples=(400, 50), cluster_std=[7.0, 2], random_state=123)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

# 모델 생성 및 학습
svc = SVC(gamma=0.05).fit(X_train, y_train)                                     

In [ ]:
mglearn.plots.plot_decision_threshold()

In [ ]:
# Classification Report - threshold: 0.5
print(classification_report(y_test, svc.predict(X_test)))

In [ ]:
# 임계값 수정된 결과 예측
pred_lower_threshold = svc.decision_function(X_test) > -0.8

In [ ]:
# Classification Report - lower threshold
print(classification_report(y_test, pred_lower_threshold))

#### 2.3.4 ROC Curve

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, svc.decision_function(X_test))
plt.plot(fpr, tpr, label='ROC Curve')
plt. xlabel('Sensitivity') 
plt. ylabel('Specificity') 

- TPR: 진짜 양성 비율
$$\text{TPR} = \frac{\text{TP}}{\text{TP} + \text{FN}}$$

- FPR: 거짓 양성 비율
$$\text{FPR} = \frac{\text{FP}}{\text{FP} + \text{TN}}$$

In [ ]:
# 데이터 로딩 및 분할

from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split

# 데이터 로딩
X, y = make_blobs(n_samples=(4000, 500), cluster_std=[7.0, 2], random_state=123)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

In [ ]:
# SVC 모델 생성 및 학습

from sklearn.svm import SVC
svc = SVC(gamma=0.05)
svc.fit(X_train, y_train)

In [ ]:
# 랜덤 포레스트 모델 생성 및 학습

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=123, max_features=2)
rf.fit(X_train, y_train)

In [ ]:
# ROC Curve - SVC

from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_test, svc.decision_function(X_test))

plt.plot(fpr, tpr, label='ROC 곡선')

plt.title('ROC Curve - SVC')
plt.xlabel('FPR')
plt.ylabel('TPR (재현율)')

close_zero = np.argmin(np.abs(thresholds))
plt.plot(fpr[close_zero], tpr[close_zero], 'o', markersize=10,
         label='임계값 0', fillstyle='none', c='k', mew=2)
plt.legend(loc=4)
plt.show()

In [ ]:
# ROC Curve - Random Forest, SVC

from sklearn.metrics import roc_curve
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, rf.predict_proba(X_test)[:, 1])

plt.plot(fpr, tpr, label='SVC의 ROC 곡선')
plt.plot(fpr_rf, tpr_rf, label='RF의 ROC 곡선')

plt.title('ROC Curve - Random Forest, SVC')
plt.xlabel('FPR')
plt.ylabel('TPR (재현율)')
plt.plot(fpr[close_zero], tpr[close_zero], 'o', markersize=10,
         label='SVC 임계값 0', fillstyle='none', c='k', mew=2)
         
close_default_rf = np.argmin(np.abs(thresholds_rf - 0.5))
plt.plot(fpr_rf[close_default_rf], tpr_rf[close_default_rf], '^', markersize=10,
         label='RF 임계값 0.5', fillstyle='none', c='k', mew=2)

plt.legend(loc=4)
plt.show()

#### 2.3.5 AUC

In [ ]:
from sklearn.metrics import roc_auc_score

svc_auc = roc_auc_score(y_test, svc.decision_function(X_test))
rf_auc  = roc_auc_score(y_test, rf.predict_proba(X_test)[:, 1])

In [ ]:
print('AUC - SVC: {:.3f}'.format(svc_auc) )

In [ ]:
print('AUC - Random Forest: {:.3f}'.format(rf_auc) )

In [ ]:
# 불균형 데이터 셋 ROC Curve

X = digits.data
y = digits.target == 9

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

for gamma in [1, 0.1, 0.01]:
    svc = SVC(gamma=gamma).fit(X_train, y_train)
    accuracy = svc.score(X_test, y_test)
    auc = roc_auc_score(y_test, svc.decision_function(X_test))
    fpr, tpr, _ = roc_curve(y_test , svc.decision_function(X_test))
    print('gamma = {:.2f}  정확도 = {:.2f}  AUC = {:.2f}'.format(gamma, accuracy, auc))
    plt.plot(fpr, tpr, label='gamma={:.2f}'.format(gamma))
    
plt.title('불균형 데이터 셋 평가')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.xlim(-0.01, 1)
plt.ylim(0, 1.02)
plt.legend(loc='best')
plt.show()

#### 2.3.6 cross_val_score

- <참고> GridSearchCV() 인자: scoring='accuracy'

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
# Accracy

acc = cross_val_score(SVC(), X, y, scoring='accuracy', cv=5) # default: accuracy

print('Accuracy')
print('교차 검증 점수:', acc)
print('교차 검증 평균 점수: {:.2f}'.format(acc.mean()))

In [ ]:
# Average precision

ap  = cross_val_score(SVC(), X, y, scoring='average_precision', cv=5)

print('Average_precision')
print('교차 검증 점수:', ap)
print('교차 검증 평균 점수: {:.2f}'.format(ap.mean()))

In [ ]:
# Recall macro

rm  = cross_val_score(SVC(), X, y, scoring='recall_macro', cv=5)

print('Recall macro')
print('교차 검증 점수:', rm)
print('교차 검증 평균 점수: {:.2f}'.format(rm.mean()))

### 2.4 다중 분류 평가 지표

In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, random_state=123)

# 모델 생성 및 학습
lr = LogisticRegression(max_iter=5000).fit(X_train, y_train)

# 예측
pred = lr.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, pred)

In [ ]:
# Confusion matrix

print('Confusion Matrix\n', confusion_matrix(y_test, pred))

In [ ]:
scores_image = mglearn.tools.heatmap(
    confusion_matrix(y_test, pred), xlabel='예측 레이블',
    ylabel='진짜 레이블', xticklabels=digits.target_names,
    yticklabels=digits.target_names, cmap=plt.cm.gray_r, fmt='%d')    
    
plt.title('오차 행렬')
plt.gca().invert_yaxis()

In [ ]:
# Classification Report

print(classification_report(y_test, pred))

In [ ]:
# f1-score

print('macro 평균 f1 점수: {:.3f}'.format(f1_score(y_test, pred, average='macro')))
print('micro 평균 f1 점수: {:.3f}'.format(f1_score(y_test, pred, average='micro')))
print('weigthted 평균 f1 점수: {:.3f}'.format(f1_score(y_test, pred, average='weighted')))

---

In [ ]:
# End of file